## Frente de pareto
Frentes de pareto acumulados hasta ahora para los distintos casos de estudio, punto seleccionado se muestra resaltado

## Evolución temporal de consumos
Comparación de los consumos de consumos entre la evolución de la serie temporal y los estimados por el modelo 

## Resultados experimentales
Gráfica del sistema completo mostrando comparativa entre las series temporales obtenidas experimentalmente y las estimadas por el modelo

## Diagrama de la instalación
Diagrama de la instalación para visualizar el punto de operación estimado por el modelo